# ADM Explained

__Pega__

__2023-03-15__

This notebook shows exactly how all the values in an ADM model report
are calculated. It also shows how the propensity is calculated for a
particular customer.

We use one of the shipped datamart exports for the example. This is a
model very similar to one used in some of the ADM PowerPoint/Excel deep
dive examples. You can change this notebook to apply to your own data.



In [ ]:
# These lines are only for rendering in the docs, and are hidden through Jupyter tags
# Do not run if you're running the notebook seperately

import plotly.io as pio

pio.renderers.default = "notebook_connected"


In [ ]:
import polars as pl
import numpy as np

import plotly.express as px
from math import log
from great_tables import GT
from pdstools import datasets
from pdstools.utils import cdh_utils

In [ ]:
model_name = "AutoNew84Months"
predictor_name = "Customer.NetWealth"
channel = "Web"

For the example we pick one particular model over a channel.
To explain the ADM model report, we use one of the active predictors as an
example. Swap for any other predictor when using different data.

In [ ]:
dm = datasets.cdh_sample()

model = dm.combined_data.filter(
    (pl.col("Name") == model_name) & (pl.col("Channel") == channel)
)

modelpredictors = (
    dm.combined_data.join(
        model.select(pl.col("ModelID").unique()), on="ModelID", how="inner"
    )
    .filter(pl.col("EntryType") != "Inactive")
    .with_columns(
        Action=pl.concat_str(["Issue", "Group"], separator="/"),
        PredictorName=pl.col("PredictorName").cast(pl.Utf8),
    )
    .collect()
)

predictorbinning = modelpredictors.filter(
    pl.col("PredictorName") == predictor_name
).sort("BinIndex")

In [ ]:
model_id = None

if (modelpredictors.select(pl.col("ModelID").unique()).shape[0] > 1) and (
    model_id is None
):
    display(
        model.group_by("ModelID")
        .agg(
            number_of_predictors=pl.col("PredictorName").n_unique(),
            model_performance=cdh_utils.weighted_performance_polars() * 100,
            response_count=pl.sum("ResponseCount"),
        )
        .collect()
        .to_pandas()
    )
    raise Exception(
        f"**{model_name}** model has multiple instances."
        "\nThis could be due to the same model name being used in different configurations, directions, issues, or having multiple treatments."
        "\nTo ensure the selection of a unique model, please choose a model_id from the table above and update the `model_id` variable at the top of this cell."
        "\nAfterward, rerun this cell."
        f"\nSee model IDs in {model_name} model above:"
    )
if model_id is not None:
    if (
        model_id
        not in modelpredictors.select(pl.col("ModelID").unique())
        .get_column("ModelID")
        .to_list()
    ):
        raise Exception(
            f"The {model_name} model does not have a model ID: {model_id}."
            f"Please ensure that the spelling of the model ID is correct."
            f"You can run `modelpredictors.select(pl.col('ModelID').unique().implode()).row(0)` to see the exact spellings of your IDs."
            "After updating the `model_id`, you can restart the notebook from the beginning."
        )

    predictors_in_selected_model = (
        modelpredictors.filter(pl.col("ModelID") == model_id)
        .select(pl.col("PredictorName").unique())
        .get_column("PredictorName")
        .to_list()
    )
    if predictor_name not in predictors_in_selected_model:
        raise Exception(
            f"{predictor_name} is not a predictor of the model with ID: {model_id}."
            "Please choose one of the available predictors below and update the **predictor_name** variable in the cell above:"
            f"\nAvailable Predictors:\n{predictors_in_selected_model}."
        )

    modelpredictors = modelpredictors.filter(pl.col("ModelID") == model_id)
    predictorbinning = predictorbinning.filter(pl.col("ModelID") == model_id)
    print(f"{model_name} model with **{model_id}** model ID is selected successfully.")

## Model Overview

The selected model is shown below. Only the currently active predictors are used for the propensity calculation, so only showing those.



In [ ]:
from great_tables import loc, style

GT(
    modelpredictors.select(
        pl.col("Action").unique(),
        pl.col("Channel").unique(),
        pl.col("Name").unique(),
        pl.col("PredictorName")
        .unique()
        .sort()
        .implode()
        .list.join(", ")
        .alias("Active Predictors"),
        (pl.col("Performance").unique() * 100).alias("Model Performance (AUC)"),
    ).transpose(include_header=True)
).tab_header("Overview").tab_options(column_labels_hidden=True).tab_style(
    style=style.text(weight="bold"), locations=loc.body(columns="column")
)

## Binning of the selected Predictor

The Model Report in Prediction Studio for this model will have a predictor binning plot like below.

All numbers can be derived from just the number of positives and negatives in each bin that are stored in the ADM Data Mart. The next sections will show exactly how that is done.

In [ ]:
display(
    GT(
        (
            predictorbinning.group_by("PredictorName")
            .agg(
                pl.first("ResponseCount").cast(pl.Utf8).alias("# Responses"),
                pl.n_unique("BinIndex").cast(pl.Utf8).alias("# Bins"),
                (pl.first("PredictorPerformance") * 100)
                .cast(pl.Utf8)
                .alias("Predictor Performance(AUC)"),
            )
            .rename({"PredictorName": "Predictor Name"})
            .transpose(include_header=True)
        )
    )
    .tab_header("Predictor information")
    .tab_options(column_labels_hidden=True)
    .tab_style(style=style.text(weight="bold"), locations=loc.body(columns="column"))
    .tab_options(table_margin_left=0)
)

fig = dm.plot.predictor_binning(
    model_id=modelpredictors.get_column("ModelID").unique().to_list()[0],
    predictor_name=predictor_name,
)
fig.update_layout(width=600, height=400)


In [ ]:
BinPositives = pl.col("BinPositives")
BinNegatives = pl.col("BinNegatives")
sumPositives = pl.sum("BinPositives")
sumNegatives = pl.sum("BinNegatives")

# TODO: add totals for first 5 columns, base rate for 6 and 0,1, see R version

binstats = predictorbinning.select(
    pl.col("BinSymbol").alias("Range/Symbol"),
    ((BinPositives + BinNegatives) / (sumPositives + sumNegatives))
    .round(3)
    .alias("Responses (%)"),
    BinPositives.alias("Positives"),
    (BinPositives / sumPositives).round(3).alias("Positives (%)"),
    BinNegatives.alias("Negatives"),
    (BinNegatives / sumNegatives).round(3).alias("Negatives (%)"),
    (BinPositives / (BinPositives + BinNegatives)).round(4).alias("Propensity (%)"),
    cdh_utils.z_ratio(neg_col=BinNegatives, pos_col=BinPositives),
    (
        (BinPositives / (BinPositives + BinNegatives))
        / (sumPositives / (BinPositives + BinNegatives).sum())
    ).alias("Lift"),
)

GT(
    binstats.vstack(
        pl.DataFrame(
            dict(
                zip(
                    binstats.columns,
                    ["Total"]
                    + [
                        binstats.select(pl.sum(col)).row(0)[0]
                        for col in binstats.columns[1:]
                    ],
                )
            ),
            schema=binstats.schema,
        )
    )
).tab_header("Binning statistics").tab_style(
    style=style.text(weight="bold"), locations=loc.body(columns="Range/Symbol")
).fmt_percent(pl.selectors.ends_with("(%)")).fmt_number(["ZRatio", "Lift"]).tab_options(
    table_margin_left=0
).tab_options(table_margin_left=0)


## Bin Statistics

### Positive and Negative ratios

Internally, ADM only keeps track of the total counts of positive and negative responses in each bin. Everything else is derived from those numbers. The percentages and totals are trivially derived, and the propensity is just the number of positives divided by the total. The numbers calculated here match the numbers from the datamart table exactly.

In [ ]:
binning_derived = predictorbinning.select(
    pl.col("BinSymbol").alias("Range/Symbol"),
    BinPositives.alias("Positives"),
    BinNegatives.alias("Negatives"),
    ((BinPositives + BinNegatives) / (sumPositives + sumNegatives)).alias(
        "Responses %"
    ),
    (BinPositives / sumPositives).alias("Positives %"),
    (BinNegatives / sumNegatives).alias("Negatives %"),
    (BinPositives / (BinPositives + BinNegatives)).round(4).alias("Propensity"),
)

pcts = ["Responses %", "Positives %", "Negatives %", "Propensity"]
GT(binning_derived).tab_header("Derived binning statistics").tab_style(
    style=style.text(weight="bold"), locations=loc.body(columns="Range/Symbol")
).tab_style(
    style=style.text(color="blue"),
    locations=loc.body(columns=pcts),
).fmt_percent(pcts).tab_options(table_margin_left=0)


### Lift

Lift is the ratio of the propensity in a particular bin over the average propensity. So a value of 1 is the average, larger than 1 means higher propensity, smaller means lower propensity:

In [ ]:
positives = pl.col("Positives")
negatives = pl.col("Negatives")
sumPositives = pl.sum("Positives")
sumNegatives = pl.sum("Negatives")
GT(
    binning_derived.select(
        "Range/Symbol",
        "Positives",
        "Negatives",
        (
            (positives / (positives + negatives))
            / (sumPositives / (positives + negatives).sum())
        )
        .round(4)
        .alias("Lift"),
    )
).tab_style(
    style=style.text(weight="bold"), locations=loc.body(columns="Range/Symbol")
).tab_style(
    style=style.text(color="blue"), locations=loc.body(columns=["Lift"])
).tab_options(table_margin_left=0)

### Z-Ratio

The Z-Ratio is also a measure of the how the propensity in a bin differs from the average, but takes into account the size of the bin and thus is statistically more relevant. It represents the number of standard deviations from the average, so centers around 0. The wider the spread, the better the predictor is.
$$\frac{posFraction-negFraction}{\sqrt(\frac{posFraction*(1-posFraction)}{\sum positives}+\frac{negFraction*(1-negFraction)}{\sum negatives})}$$ 

See the calculation here, which is also included in [cdh_utils' zRatio()](https://pegasystems.github.io/pega-datascientist-tools/Python/autoapi/pdstools/utils/cdh_utils/index.html#pdstools.utils.cdh_utils.zRatio).

In [ ]:
def z_ratio(
    pos_col: pl.Expr = pl.col("BinPositives"), neg_col: pl.Expr = pl.col("BinNegatives")
) -> pl.Expr:
    def get_fracs(pos_col=pl.col("BinPositives"), neg_col=pl.col("BinNegatives")):
        return pos_col / pos_col.sum(), neg_col / neg_col.sum()

    def z_ratio_impl(
        pos_fraction_col=pl.col("posFraction"),
        neg_fraction_col=pl.col("negFraction"),
        positives_col=pl.sum("BinPositives"),
        negatives_col=pl.sum("BinNegatives"),
    ):
        return (
            (pos_fraction_col - neg_fraction_col)
            / (
                (pos_fraction_col * (1 - pos_fraction_col) / positives_col)
                + (neg_fraction_col * (1 - neg_fraction_col) / negatives_col)
            ).sqrt()
        ).alias("ZRatio")

    return z_ratio_impl(*get_fracs(pos_col, neg_col), pos_col.sum(), neg_col.sum())


GT(
    binning_derived.select(
        "Range/Symbol", "Positives", "Negatives", "Positives %", "Negatives %"
    ).with_columns(z_ratio(positives, negatives).round(4))
).tab_style(
    style=style.text(weight="bold"), locations=loc.body(columns="Range/Symbol")
).tab_style(
    style=style.text(color="blue"), locations=loc.body(columns=["ZRatio"])
).fmt_percent(pl.selectors.ends_with("%")).tab_options(table_margin_left=0)

## Predictor AUC


The predictor AUC is the univariate performance of this predictor against the outcome. This too can be derived from the positives and negatives and
there is  a convenient function in pdstools to calculate it directly from the positives and negatives.

This function is implemented in cdh_utils: [cdh_utils.auc_from_bincounts()](https://pegasystems.github.io/pega-datascientist-tools/Python/autoapi/pdstools/utils/cdh_utils/index.html#pdstools.utils.cdh_utils.auc_from_bincounts).

In [ ]:
pos = binning_derived.get_column("Positives")
neg = binning_derived.get_column("Negatives")
probs = binning_derived.get_column("Propensity")
order = probs.arg_sort()
FPR = pl.Series([0.0]).extend(neg[order].cum_sum() / neg[order].sum())
TPR = pl.Series([0.0]).extend(pos[order].cum_sum() / pos[order].sum())
if TPR[1] < 1 - FPR[1]:
    FPR, TPR = TPR, FPR


In [ ]:
pos = binning_derived.get_column("Positives").to_numpy()
neg = binning_derived.get_column("Negatives").to_numpy()
probs = binning_derived.get_column("Propensity").to_numpy()
order = np.argsort(probs)

FPR = np.cumsum(neg[order]) / np.sum(neg[order])
TPR = np.cumsum(pos[order]) / np.sum(pos[order])
TPR = np.insert(TPR, 0, 0, axis=0)
FPR = np.insert(FPR, 0, 0, axis=0)
# Checking whether classifier labels are correct
if TPR[1] < 1 - FPR[1]:
    temp = FPR
    FPR = TPR
    TPR = temp
auc = cdh_utils.auc_from_bincounts(pos=pos, neg=neg, probs=probs)

fig = px.line(
    x=[1 - x for x in FPR],
    y=TPR,
    labels=dict(x="Specificity", y="Sensitivity"),
    title=f"AUC = {auc.round(3)}",
    width=700,
    height=700,
    range_x=[1, 0],
    template="none",
)
fig.add_shape(type="line", line=dict(dash="dash"), x0=1, x1=0, y0=0, y1=1)
fig.show()


## Naive Bayes and Log Odds

The basis for the Naive Bayes algorithm is Bayes' Theorem:

$$p(C_k|x) = \frac{p(x|C_k)*p(C_k)}{p(x)}$$

with $C_k$ the outcome and $x$ the customer. Bayes' theorem turns the
question "what's the probability to accept this action given a customer" around to 
"what's the probability of this customer given an action". With the independence
assumption, and after applying a log odds transformation we get a log odds score 
that can be calculated efficiently and in a numerically stable manner:

$$log\ odds\ score = \sum_{p\ \in\ active\ predictors}log(p(x_p|Positive)) + log(p_{positive}) - \sum_plog(p(x_p|Negative)) - log(p_{negative})$$
note that the _prior_ can be written as:

$$log(p_{positive}) - log(p_{negative}) = log(\frac{TotalPositives}{Total})-log(\frac{TotalNegatives}{Total}) = log(TotalPositives) - log(TotalNegatives)$$


## Predictor Contribution

The contribution (_conditional log odds_) of an active predictor $p$ for bin $i$ with the number
of positive and negative responses in $Positives_i$ and $Negatives_i$ is calculated as (note the "laplace smoothing" to avoid log 0 issues):

$$contribution_p = \log(Positives_i+\frac{1}{nBins}) - \log(Negatives_i+\frac{1}{nBins}) - \log(1+\sum_{i\ = 1..nBins}{Positives_i}) + \log(1+\sum_i{Negatives_i})$$


In [ ]:
N = binning_derived.shape[0]
GT(
    binning_derived.with_columns(
        LogOdds=(pl.col("Positives %") / pl.col("Negatives %")).log().round(5),
        ModifiedLogOdds=(
            ((positives + 1 / N).log() - (positives.sum() + 1).log())
            - ((negatives + 1 / N).log() - (negatives.sum() + 1).log())
        ).round(5),
    ).drop("Responses %", "Propensity")
).tab_style(
    style=style.text(weight="bold"), locations=loc.body(columns="Range/Symbol")
).tab_style(
    style=style.text(color="blue"),
    locations=loc.body(columns=["LogOdds", "ModifiedLogOdds"]),
).fmt_percent(pl.selectors.ends_with("%")).tab_options(table_margin_left=0)

## Propensity mapping

### Log odds contribution for all the predictors

The final score is loosely referred to as "the average contribution" but
in fact is a little more nuanced. The final score is calculated as:

$$score = \frac{\log(1 + TotalPositives) – \log(1 + TotalNegatives) + \sum_p contribution_p}{1 + nActivePredictors}$$

Here, $TotalPositives$ and $TotalNegatives$ are the total number of
positive and negative responses to the model.

Below an example. From all the active predictors of the model 
we pick a value (in the middle for numerics, first symbol
for symbolics) and show the (modified) log odds. The final score is
calculated per the above formula, and this is the value that is mapped
to a propensity value by the classifier (which is constructed using the
[PAV(A)](https://en.wikipedia.org/wiki/Isotonic_regression) algorithm).


In [ ]:
def middle_bin():
    return pl.col("BinIndex") == (pl.max("BinIndex") / 2).floor().cast(pl.UInt32)


if not all(
    col in modelpredictors.columns for col in ["BinLowerBound", "BinUpperBound"]
):

    def extract_numbers_in_contents(s: str, index):
        numbers = re.findall(r"[-+]?\d*\.\d+|\d+", s)
        try:
            number = float(numbers[index])
        except:
            number = 0
        return number

    modelpredictors = modelpredictors.with_columns(
        pl.col("Contents").cast(pl.Utf8)
    ).with_columns(
        pl.when(pl.col("Type") == "numeric")
        .then(
            pl.col("Contents").map_elements(
                lambda col: extract_numbers_in_contents(col, 0)
            )
        )
        .otherwise(pl.lit(-9999))
        .alias("BinLowerBound")
        .cast(pl.Float32),
        pl.when(pl.col("Type") == "numeric")
        .then(
            pl.col("Contents").map_elements(
                lambda col: extract_numbers_in_contents(col, 1)
            )
        )
        .otherwise(pl.lit(-9999))
        .alias("BinUpperBound")
        .cast(pl.Float32),
    )


def row_wise_log_odds(bin, positives, negatives):
    bin, N = bin.list.get(0) - 1, positives.list.len()
    pos, neg = positives.list.get(bin), negatives.list.get(bin)
    pos_sum, neg_sum = positives.list.sum(), negatives.list.sum()
    return (
        ((pos + (1 / N)).log() - (pos_sum + 1).log())
        - (((neg + (1 / N)).log()) - (neg_sum + 1).log())
    ).alias("Modified Log odds")


df = (
    modelpredictors.filter(pl.col("PredictorName") != "Classifier")
    .group_by("PredictorName")
    .agg(
        Value=pl.when(pl.col("Type").first() == "numeric")
        .then(
            ((pl.col("BinLowerBound") + pl.col("BinUpperBound")) / 2).filter(
                middle_bin()
            )
        )
        .otherwise(
            pl.col("BinSymbol").str.split(",").list.first().filter(middle_bin())
        ),
        Bin=pl.col("BinIndex").filter(middle_bin()),
        Positives=pl.col("BinPositives"),
        Negatives=pl.col("BinNegatives"),
    )
    .with_columns(
        pl.col(["Positives", "Negatives"]).list.get(pl.col("Bin").list.get(0) - 1),
        pl.col("Bin", "Value").list.get(0),
        LogOdds=row_wise_log_odds(
            pl.col("Bin"), pl.col("Positives"), pl.col("Negatives")
        ).round(4),
    )
    .sort("PredictorName")
)

classifier = (
    modelpredictors.filter(pl.col("EntryType") == "Classifier")
    .with_columns(
        Propensity=(BinPositives / (BinPositives / BinNegatives)),
        AdjustedPropensity=((0.5 + BinPositives) / (1 + BinPositives + BinNegatives)),
        ZRatio=cdh_utils.z_ratio(neg_col=BinNegatives, pos_col=BinPositives).round(4),
        Lift=(
            (BinPositives / (BinPositives + BinNegatives))
            / (sumPositives / (BinPositives + BinNegatives).sum())
        ),
    )
    .select(
        [
            pl.col("BinIndex").alias("Index"),
            pl.col("BinSymbol").alias("Bin"),
            BinPositives.alias("Positives"),
            BinNegatives.alias("Negatives"),
            (pl.cum_sum("BinResponseCount") / pl.sum("BinResponseCount")).alias(
                "Cum. Total (%)"
            ),
            (pl.col("BinPropensity")).alias("Propensity (%)"),
            (pl.col("AdjustedPropensity")).alias("Adjusted Propensity (%)"),
            (pl.cum_sum("BinPositives") / pl.sum("BinPositives")).alias(
                "Cum Positives (%)"
            ),
            pl.col("ZRatio"),
            pl.col("Lift").alias("Lift(%)"),
            pl.col("BinResponseCount").alias("Responses"),
        ]
    )
)
classifierLogOffset = log(1 + classifier["Positives"].sum()) - log(
    1 + classifier["Negatives"].sum()
)

propensity_mapping = df.vstack(
    pl.DataFrame(
        dict(
            zip(
                df.columns,
                ["Final Score"]
                + [None] * 4
                + [(df["LogOdds"].sum() + classifierLogOffset) / (len(df) + 1)],
            )
        ),
        schema=df.schema,
    )
)

GT(propensity_mapping).tab_style(
    style=style.text(weight="bold"), locations=loc.body(columns="PredictorName")
).tab_style(
    style=style.text(color="blue"), locations=loc.body(columns=["LogOdds"])
).tab_options(table_margin_left=0)

## Classifier

The success rate is defined as $\frac{positives}{positives+negatives}$ per bin. 

The adjusted propensity that is returned is a small modification (Laplace smoothing) to this and calculated as $\frac{0.5+positives}{1+positives+negatives}$ so empty models return a propensity of 0.5.


In [ ]:
# TODO see if we can port the "getActiveRanges" code to python so to highlight the classifier rows that are "active"

GT(classifier.drop("Responses")).tab_style(
    style=style.text(weight="bold"), locations=loc.body(columns="Index")
).tab_style(
    style=style.text(color="blue"),
    locations=loc.body(columns=["Adjusted Propensity (%)"]),
).fmt_percent(pl.selectors.ends_with("(%)")).tab_options(table_margin_left=0)

## Final Propensity

Below the classifier mapping. On the x-axis the binned scores (log odds values), on the y-axis the Propensity. Note the returned propensities are following a slightly adjusted formula, see the table above. The bin that contains the calculated final score is highlighted.

In [ ]:
score = propensity_mapping.filter(PredictorName="Final Score")["LogOdds"][0]
score_bin = (
    modelpredictors.filter(pl.col("EntryType") == "Classifier")
    .select(
        pl.col("BinSymbol").filter(
            pl.lit(score).is_between(pl.col("BinLowerBound"), pl.col("BinUpperBound"))
        )
    )
    .item()
)
score_responses = modelpredictors.filter(
    (pl.col("EntryType") == "Classifier") & (pl.col("BinSymbol") == score_bin)
)["BinResponseCount"][0]

score_bin_index = (
    modelpredictors.filter(pl.col("EntryType") == "Classifier")["BinSymbol"]
    .to_list()
    .index(score_bin)
)

score_propensity = classifier.row(score_bin_index, named=True)[
    "Adjusted Propensity (%)"
]

adjusted_propensity = (
    modelpredictors.filter(pl.col("EntryType") == "Classifier")
    .with_columns(
        AdjustedPropensity=(
            (0.5 + BinPositives) / (1 + BinPositives + BinNegatives)
        ).round(5),
    )
    .select(
        pl.col("AdjustedPropensity").filter(
            (pl.col("BinLowerBound") < score) & (pl.col("BinUpperBound") > score)
        )
    )["AdjustedPropensity"][0]
)
fig = dm.plot.score_distribution(
    model_id=modelpredictors.get_column("ModelID").unique().to_list()[0]
).add_annotation(
    x=score_bin,
    y=score_propensity / 100,
    text=f"Returned propensity: {score_propensity*100:.2f}%",
    bgcolor="#FFFFFF",
    bordercolor="#000000",
    showarrow=False,
    yref="y2",
    opacity=0.7,
)
bin_index = list(fig.data[0]["x"]).index(score_bin)
fig.data[0]["marker_color"] = (
    ["grey"] * bin_index
    + ["#1f77b4"]
    + ["grey"] * (classifier.shape[0] - bin_index - 1)
)
fig